## Создайте RDD из файла на HDFS
Файл расположен на HDFS по пути hdfs://user/atitov/data1.json

In [9]:
rdd = sc.textFile("hdfs:///user/atitov/data1.json")

## Получите список всех ключей

In [2]:
import json

def parse_json(data):
    try:
        parsed = json.loads(data)
    except json.decoder.JSONDecodeError:
        parsed = {}
    return parsed

keys = rdd.flatMap(lambda x: parse_json(x).keys()).distinct().collect()
print(keys)

['name', 'country', 'continent', 'population']


## Исключите дубликаты из RDD
Дубликатом считаются записи с одинаковыми значениями полей name и country

In [3]:
unique = rdd\
    .map(lambda x: parse_json(x))\
    .map(lambda x: ((x.get('name'), x.get('country')), x))\
    .reduceByKey(lambda x,y: x)\
    .map(lambda x: x[1])

unique.collect()

[{'name': 'Moscow',
  'country': 'Russia',
  'continent': 'Europe',
  'population': 12380664},
 {'name': 'Madrid', 'country': 'Spain'},
 {'name': 'Berlin',
  'country': 'Germany',
  'continent': 'Europe',
  'population': 3490105},
 {'name': 'Cairo',
  'country': 'Egypt',
  'continent': 'Africa',
  'population': 11922948},
 {},
 {'name': 'Paris',
  'country': 'France',
  'continent': 'Europe',
  'population': 2196936},
 {'name': 'Barselona', 'country': 'Spain', 'continent': 'Europe'}]

## Удалите нулевые элементы

In [4]:
valuable = unique.filter(lambda x: x)
valuable.collect()

[{'name': 'Moscow',
  'country': 'Russia',
  'continent': 'Europe',
  'population': 12380664},
 {'name': 'Madrid', 'country': 'Spain'},
 {'name': 'Berlin',
  'country': 'Germany',
  'continent': 'Europe',
  'population': 3490105},
 {'name': 'Cairo',
  'country': 'Egypt',
  'continent': 'Africa',
  'population': 11922948},
 {'name': 'Paris',
  'country': 'France',
  'continent': 'Europe',
  'population': 2196936},
 {'name': 'Barselona', 'country': 'Spain', 'continent': 'Europe'}]

## Найдите город с самым большим населением

In [5]:
max_population = valuable.max(key=lambda x: x.get('population', 0))
print(max_population['name'])

Moscow


## Посчитайте топ-2 континентов по населению

In [6]:
max_continent_pop = \
    valuable\
        .map(lambda x: (x.get('continent', 'Earth'), x.get('population', 0)))\
        .reduceByKey(lambda x,y: x + y)\
        .sortByKey(ascending=False, keyfunc=lambda x: x[1]).take(2)
    
print(max_continent_pop)

[('Europe', 18067705), ('Africa', 11922948)]


## Добавьте к каждой записи новое поле
Имя поля - populous. Значение - true, если континент является первым в топ-2 списке и false - в остальных случаях

In [7]:
populous_bc = sc.broadcast(max_continent_pop[0][0])

def add_key(d, k, v):
    d[k] = v
    return d

valuable.map(lambda x: add_key(x, 'populous', x.get('continent') == populous_bc.value)).collect()

[{'name': 'Moscow',
  'country': 'Russia',
  'continent': 'Europe',
  'population': 12380664,
  'populous': True},
 {'name': 'Madrid', 'country': 'Spain', 'populous': False},
 {'name': 'Berlin',
  'country': 'Germany',
  'continent': 'Europe',
  'population': 3490105,
  'populous': True},
 {'name': 'Cairo',
  'country': 'Egypt',
  'continent': 'Africa',
  'population': 11922948,
  'populous': False},
 {'name': 'Paris',
  'country': 'France',
  'continent': 'Europe',
  'population': 2196936,
  'populous': True},
 {'name': 'Barselona',
  'country': 'Spain',
  'continent': 'Europe',
  'populous': True}]

## Измените партиционирование RDD по значению поля continent

In [8]:
import mmh3

def new_partitioner(key):
    return mmh3.hash(key)

valuable\
    .map(lambda x: (x.get('continent', 'Earth'), x))\
    .repartitionAndSortWithinPartitions(3, new_partitioner, True)\
    .map(lambda x: x[1])\
    .glom().collect()


[[{'name': 'Madrid', 'country': 'Spain'}],
 [{'name': 'Cairo',
   'country': 'Egypt',
   'continent': 'Africa',
   'population': 11922948},
  {'name': 'Moscow',
   'country': 'Russia',
   'continent': 'Europe',
   'population': 12380664},
  {'name': 'Berlin',
   'country': 'Germany',
   'continent': 'Europe',
   'population': 3490105},
  {'name': 'Paris',
   'country': 'France',
   'continent': 'Europe',
   'population': 2196936},
  {'name': 'Barselona', 'country': 'Spain', 'continent': 'Europe'}],
 []]